### Imports + Projektordner

In [1]:
import os
import time
import random
import glob

import pandas as pd
import geopandas as gpd
import osmnx as ox
import requests

import folium
from folium.plugins import HeatMap

# -------------------------
# Projektordner (alles an einem Ort)
# -------------------------
PROJECT_DIR = "scs_osm_project"   # <- kannst du umbenennen
CSV_PATH = "islands_update.csv"          # <- dein CSV-Dateiname (Semikolon)

CACHE_DIR  = os.path.join(PROJECT_DIR, "cache_gpkg")
MASTER_DIR = os.path.join(PROJECT_DIR, "master_gpkg")
HTML_DIR   = os.path.join(PROJECT_DIR, "heatmaps_html")

for d in [CACHE_DIR, MASTER_DIR, HTML_DIR]:
    os.makedirs(d, exist_ok=True)

print("Folders:")
print("CACHE :", CACHE_DIR)
print("MASTER:", MASTER_DIR)
print("HTML  :", HTML_DIR)


Folders:
CACHE : scs_osm_project\cache_gpkg
MASTER: scs_osm_project\master_gpkg
HTML  : scs_osm_project\heatmaps_html


### CSV laden + bbox erzeugen (ohne Fallback, fehlende bbox ignorieren)

In [2]:
df = pd.read_csv(CSV_PATH, sep=";", engine="python")

needed = ["id", "island", "island_group", "country", "north", "south", "west", "east"]
missing = [c for c in needed if c not in df.columns]
if missing:
    raise ValueError(f"Fehlende Spalten in CSV: {missing}\nGefundene Spalten: {list(df.columns)}")

df = df[needed].copy()

# Zahlen robust parsen
for c in ["north", "south", "west", "east"]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Zeilen ohne vollständige bbox ignorieren
mask_ok = df[["north", "south", "west", "east"]].notna().all(axis=1)
skipped = df.loc[~mask_ok, ["id", "island"]].copy()
df = df.loc[mask_ok].copy()

print("Rows used (with bbox):", len(df))
print("Rows skipped (missing bbox):", len(skipped))
if len(skipped) > 0:
    display(skipped.head(20))

# OSMnx bbox order: (west, south, east, north)
df["bbox"] = list(zip(df["west"], df["south"], df["east"], df["north"]))

df.head()


Rows used (with bbox): 70
Rows skipped (missing bbox): 0


id            island island_group country   north  south    west    east  \
0   1    Cuarteron Reef      Spratly   China   8.890   8.84  112.80  112.87   
1   2  Fiery Cross Reef      Spratly   China   9.580   9.53  112.86  112.92   
2   3       Gaven Reefs      Spratly   China  10.220  10.19  114.21  114.24   
3   4       Hughes Reef      Spratly   China   9.927   9.90  114.48  114.51   
4   5      Johnson Reef      Spratly   China   9.750   9.69  114.26  114.30   

                             bbox  
0     (112.8, 8.84, 112.87, 8.89)  
1    (112.86, 9.53, 112.92, 9.58)  
2  (114.21, 10.19, 114.24, 10.22)  
3    (114.48, 9.9, 114.51, 9.927)  
4     (114.26, 9.69, 114.3, 9.75)

### QUERIES + Overpass/OSMnx Settings + Rate limit

In [3]:
# "schöne" Typ-Namen -> werden Dateinamen & Master-Layer
QUERIES = {
    "runways": {"aeroway": "runway"},
    "helipads": {"aeroway": ["helipad", "heliport"]},
    "towers": {"man_made": "tower"},
    "marinas": {"leisure": "marina"},
    "ferry_terminals": {"amenity": "ferry_terminal"},
}

# OSMnx/Overpass Setup
ox.settings.use_cache = True
ox.settings.log_console = True
ox.settings.overpass_settings = "[out:json][timeout:180]"  # ohne ';'
ox.settings.overpass_url = "https://overpass.kumi.systems/api/interpreter"
# Alternativen:
# ox.settings.overpass_url = "https://overpass.nchc.org.tw/api/interpreter"
# ox.settings.overpass_url = "https://overpass.openstreetmap.ru/api/interpreter"

# Rate-Limit: für viele Inseln lieber konservativ (weniger Drosselung)
BASE_SLEEP = 6
JITTER = 2

def safe_sleep():
    time.sleep(BASE_SLEEP + random.uniform(0, JITTER))

print("Overpass URL:", ox.settings.overpass_url)
print("Queries:", list(QUERIES.keys()))
print("Islands to process:", len(df))


Overpass URL: https://overpass.kumi.systems/api/interpreter
Queries: ['runways', 'helipads', 'towers', 'marinas', 'ferry_terminals']
Islands to process: 70


### Helper-Funktionen (exists-check + Dateipfade + fetch)

In [4]:
def bbox_to_overpass_str(bbox):
    # bbox: (west,south,east,north) -> overpass: (south,west,north,east)
    west, south, east, north = bbox
    return f"({south},{west},{north},{east})"

def overpass_exists(bbox, tags, timeout=90):
    """Schneller Check: True wenn >=1 Element existiert."""
    bbox_str = bbox_to_overpass_str(bbox)

    selectors = []
    for k, v in tags.items():
        if isinstance(v, (list, tuple, set)):
            for vv in v:
                selectors.append((k, vv))
        else:
            selectors.append((k, v))

    parts = []
    for k, v in selectors:
        parts.append(f'node["{k}"="{v}"]{bbox_str};')
        parts.append(f'way["{k}"="{v}"]{bbox_str};')
        parts.append(f'relation["{k}"="{v}"]{bbox_str};')

    query = f"""
    [out:json][timeout:{timeout}];
    (
      {''.join(parts)}
    );
    out ids qt 1;
    """

    r = requests.post(ox.settings.overpass_url, data={"data": query}, timeout=timeout + 30)
    r.raise_for_status()
    data = r.json()
    return len(data.get("elements", [])) > 0

def slug(s: str) -> str:
    """Dateiname sicher machen (keine Leerzeichen/Sonderzeichen)."""
    return "".join(ch if ch.isalnum() else "_" for ch in str(s)).strip("_")

def cache_path(island_id, island_name, layer_key):
    # pro Insel × Typ eine Cache-Datei
    return os.path.join(CACHE_DIR, f"{int(island_id):03d}__{slug(island_name)}__{layer_key}.gpkg")

def fetch_features_bbox(bbox, tags):
    # OSMnx bbox=(west,south,east,north)
    return ox.features_from_bbox(bbox=bbox, tags=tags).reset_index()


### Download-Funktion (pro Insel in Cache speichern)

In [5]:
def download_island_to_cache(island_id, island_name, country, island_group, bbox, overwrite=False):
    """
    Lädt alle QUERIES für eine Insel und speichert pro Typ eine GPKG in CACHE_DIR.
    - wenn Cache existiert und overwrite=False -> skip
    - exists-check (schnell)
    - nur wenn exists=True: Geometrie laden und speichern
    """
    print(f"\n=== {int(island_id):03d} | {island_name} ({country}) bbox={bbox} ===")

    for layer_key, tags in QUERIES.items():
        out_path = cache_path(island_id, island_name, layer_key)

        # Cache nutzen
        if (not overwrite) and os.path.exists(out_path):
            try:
                g_cached = gpd.read_file(out_path)
                print(f"  {layer_key}: cached ({len(g_cached)})")
                safe_sleep()
                continue
            except Exception:
                pass

        # exists-check (einmal, plus 1 Retry bei echtem Serverproblem)
        try:
            exists = overpass_exists(bbox, tags, timeout=90)
        except Exception as e:
            print(f"  {layer_key}: exists-check error -> retry once | {repr(e)}")
            safe_sleep()
            exists = overpass_exists(bbox, tags, timeout=120)

        print(f"  {layer_key}: exists={exists}")

        if not exists:
            safe_sleep()
            continue

        # Download (1 Versuch, sonst skip)
        try:
            t0 = time.time()
            g = fetch_features_bbox(bbox, tags)
            dt = time.time() - t0
            print(f"  {layer_key}: downloaded {len(g)} in {dt:.1f}s")

            # Metadaten
            g["src_island_id"] = int(island_id)
            g["src_island"] = island_name
            g["src_country"] = country
            g["src_group"] = island_group
            g["src_layer"] = layer_key

            g.to_file(out_path, driver="GPKG")
            print(f"  {layer_key}: saved -> {os.path.basename(out_path)}")
        except Exception as e:
            print(f"  {layer_key}: download error (skipped) -> {repr(e)}")

        safe_sleep()


### RUN: alle Inseln downloaden (Cache füllen)

In [7]:
# overwrite=False -> nutzt Cache, lädt nicht neu
# overwrite=True  -> lädt alles neu (vorsicht: viele Requests)

overwrite = True

START_ID = 26      # <-- HIER ändern (inklusive)
END_ID   = 26     # <-- HIER ändern (inklusive)
# Tipp: END_ID = None -> bis zum Ende laufen lassen

# optional aber sinnvoll: nach id sortieren
df_run = df.sort_values("id").copy()

# ab START_ID weiterlaufen (inkl.)
df_run = df_run[df_run["id"] >= START_ID].copy()

# bis END_ID laufen (inkl.), falls gesetzt
if END_ID is not None:
    df_run = df_run[df_run["id"] <= END_ID].copy()

total = len(df_run)
for i, row in enumerate(df_run.itertuples(index=False), start=1):
    print(f"\n[{i}/{total}] Processing: id={row.id} | {row.island}")
    download_island_to_cache(
        island_id=row.id,
        island_name=row.island,
        country=row.country,
        island_group=row.island_group,
        bbox=row.bbox,
        overwrite=overwrite
    )

print("\nDone downloading selected islands.")



[1/1] Processing: id=26 | Yagong Island

=== 026 | Yagong Island (China) bbox=(111.634, 16.564, 111.688, 16.569) ===
  runways: exists=False
  helipads: exists=False
  towers: exists=False
  marinas: exists=False
  ferry_terminals: exists=True
  ferry_terminals: downloaded 1 in 88.3s
  ferry_terminals: saved -> 026__Yagong_Island__ferry_terminals.gpkg

Done downloading selected islands.


### Master global (pro Typ eine Datei)

In [8]:
def build_master_global():
    """
    Liest ALLE Cache-GPKGs aus CACHE_DIR und schreibt pro Typ eine Master-GPKG nach MASTER_DIR.
    Vorteil: Heatmaps/Analysen müssen dann nur noch wenige Dateien lesen.
    """
    files = sorted(glob.glob(os.path.join(CACHE_DIR, "*.gpkg")))
    print("Total cache files:", len(files))
    if not files:
        print("No cache files found.")
        return

    buckets = {k: [] for k in QUERIES.keys()}

    for fp in files:
        base = os.path.splitext(os.path.basename(fp))[0]
        # erwartet: 002__Fiery_Cross_Reef__helipads
        parts = base.split("__")
        if len(parts) < 3:
            continue
        layer_key = parts[-1]
        if layer_key not in buckets:
            continue

        g = gpd.read_file(fp)
        if len(g) == 0:
            continue

        if "src_layer" not in g.columns:
            g["src_layer"] = layer_key

        buckets[layer_key].append(g)

    for layer_key, parts in buckets.items():
        out_master = os.path.join(MASTER_DIR, f"master__{layer_key}.gpkg")

        if not parts:
            print(f"Master {layer_key}: no data")
            continue

        g_all = gpd.GeoDataFrame(pd.concat(parts, ignore_index=True), geometry="geometry", crs=parts[0].crs)

        # einfache Dedup-Option
        if "osmid" in g_all.columns:
            before = len(g_all)
            g_all = g_all.drop_duplicates(subset=["osmid", "src_layer"], keep="first")
            print(f"Master {layer_key}: dedup {before} -> {len(g_all)}")

        g_all.to_file(out_master, driver="GPKG")
        print(f"Master {layer_key}: saved {os.path.basename(out_master)} ({len(g_all)})")

build_master_global()


Total cache files: 98
Master runways: saved master__runways.gpkg (9)
Master helipads: saved master__helipads.gpkg (51)
Master towers: saved master__towers.gpkg (189)
Master marinas: saved master__marinas.gpkg (19)
Master ferry_terminals: saved master__ferry_terminals.gpkg (21)


### Heatmaps (pro Typ HTML)

In [10]:
def to_points(gdf):
    """
    Heatmap braucht Punkte:
    - Points bleiben Points
    - Linien/Polygone -> centroid (Näherung)
    """
    g = gdf[gdf.geometry.notna()].copy()
    g["point_geom"] = g.geometry.apply(lambda geom: geom if geom.geom_type == "Point" else geom.centroid)
    g["lon"] = g["point_geom"].x
    g["lat"] = g["point_geom"].y
    return g.dropna(subset=["lat", "lon"])

def make_heatmap_from_points(points_df, out_html, zoom_start=5):
    if len(points_df) == 0:
        print("No points ->", out_html)
        return None

    center = [points_df["lat"].mean(), points_df["lon"].mean()]
    m = folium.Map(location=center, zoom_start=zoom_start, tiles="CartoDB positron")

    HeatMap(
        data=points_df[["lat", "lon"]].values.tolist(),
        radius=18, blur=14, min_opacity=0.25
    ).add_to(m)

    m.save(out_html)
    print("Saved:", out_html)
    return m

for layer_key in QUERIES.keys():
    master_path = os.path.join(MASTER_DIR, f"master__{layer_key}.gpkg")
    out_html = os.path.join(HTML_DIR, f"heatmap__{layer_key}.html")

    if not os.path.exists(master_path):
        print("Missing master:", master_path)
        continue

    g = gpd.read_file(master_path)
    pts = to_points(g)

    # Zoom anpassen:
    # - Südchinesisches Meer: z.B. 5 oder 6
    make_heatmap_from_points(pts, out_html, zoom_start=5)


Saved: scs_osm_project\heatmaps_html\heatmap__runways.html
Saved: scs_osm_project\heatmaps_html\heatmap__helipads.html
Saved: scs_osm_project\heatmaps_html\heatmap__towers.html
Saved: scs_osm_project\heatmaps_html\heatmap__marinas.html
Saved: scs_osm_project\heatmaps_html\heatmap__ferry_terminals.html


### Neuer Workflow: Vorbereitung für ArcGIS Online (alle Masterdateien auf einmal)

In [9]:
import geopandas as gpd
from pathlib import Path

MASTER_DIR = Path(r"scs_osm_project/master_gpkg")

# Alle master__*.gpkg finden, aber NICHT die bereits erzeugten Outputs
master_files = sorted(MASTER_DIR.glob("master__*.gpkg"))
master_files = [
    p for p in master_files
    if "_POINTS_FOR_AGOL" not in p.name and "_PUBLISH_READY" not in p.name
]

print("Found master GPKGs:")
for p in master_files:
    print(" -", p)

for gpkg_path in master_files:
    print("\n==============================")
    print("FILE:", gpkg_path.name)
    print("==============================")

    gdf = gpd.read_file(gpkg_path)

    print("Rows:", len(gdf))
    print(gdf.geom_type.value_counts(dropna=False))
    print("CRS:", gdf.crs)


Found master GPKGs:
 - scs_osm_project\master_gpkg\master__ferry_terminals.gpkg
 - scs_osm_project\master_gpkg\master__helipads.gpkg
 - scs_osm_project\master_gpkg\master__marinas.gpkg
 - scs_osm_project\master_gpkg\master__runways.gpkg
 - scs_osm_project\master_gpkg\master__towers.gpkg

FILE: master__ferry_terminals.gpkg
Rows: 21
Point           13
Polygon          6
MultiPolygon     2
Name: count, dtype: int64
CRS: EPSG:4326

FILE: master__helipads.gpkg
Rows: 51
Polygon    42
Point       9
Name: count, dtype: int64
CRS: EPSG:4326

FILE: master__marinas.gpkg
Rows: 19
Polygon    15
Point       4
Name: count, dtype: int64
CRS: EPSG:4326

FILE: master__runways.gpkg
Rows: 9
LineString    9
Name: count, dtype: int64
CRS: EPSG:4326

FILE: master__towers.gpkg
Rows: 189
Point      110
Polygon     79
Name: count, dtype: int64
CRS: EPSG:4326


### ArcGIS taugliche Felder

In [10]:
import geopandas as gpd
from pathlib import Path

MASTER_DIR = Path(r"scs_osm_project/master_gpkg")

master_files = sorted(MASTER_DIR.glob("master__*.gpkg"))
master_files = [
    p for p in master_files
    if "_POINTS_FOR_AGOL" not in p.name and "_PUBLISH_READY" not in p.name
]

def geom_to_point(geom):
    if geom is None:
        return None
    gt = geom.geom_type
    if gt == "Point":
        return geom
    if gt in ("Polygon", "MultiPolygon"):
        return geom.representative_point()
    if gt in ("LineString", "MultiLineString"):
        try:
            return geom.interpolate(0.5, normalized=True)
        except Exception:
            return geom.centroid
    return geom.centroid

for input_gpkg in master_files:
    layer_key = input_gpkg.stem.replace("master__", "")  # z.B. "towers", "helipads", ...
    output_gpkg = MASTER_DIR / f"master__{layer_key}_POINTS_FOR_AGOL.gpkg"
    layer_name = f"{layer_key}_points"

    print("\n==============================")
    print("INPUT :", input_gpkg.name)
    print("OUTPUT:", output_gpkg.name)
    print("LAYER :", layer_name)
    print("==============================")

    gdf = gpd.read_file(input_gpkg).copy()

    # nur gültige/ vorhandene Geometrie
    gdf = gdf[gdf.geometry.notna()].copy()
    gdf = gdf[~gdf.geometry.is_empty].copy()

    # CRS auf WGS84
    if gdf.crs is None:
        gdf = gdf.set_crs(epsg=4326)   # nur wenn du sicher bist, dass es schon lon/lat ist
    else:
        gdf = gdf.to_crs(epsg=4326)

    # -> Point-Geometrie
    gdf["geometry"] = gdf.geometry.apply(geom_to_point)

    # nochmal Nulls raus
    gdf = gdf[gdf.geometry.notna()].copy()
    gdf = gdf[~gdf.geometry.is_empty].copy()

    print("After conversion:", len(gdf))
    print(gdf.geom_type.value_counts())

    gdf.to_file(output_gpkg, layer=layer_name, driver="GPKG")
    print("Wrote:", output_gpkg)



INPUT : master__ferry_terminals.gpkg
OUTPUT: master__ferry_terminals_POINTS_FOR_AGOL.gpkg
LAYER : ferry_terminals_points
After conversion: 21
Point    21
Name: count, dtype: int64
Wrote: scs_osm_project\master_gpkg\master__ferry_terminals_POINTS_FOR_AGOL.gpkg

INPUT : master__helipads.gpkg
OUTPUT: master__helipads_POINTS_FOR_AGOL.gpkg
LAYER : helipads_points
After conversion: 51
Point    51
Name: count, dtype: int64
Wrote: scs_osm_project\master_gpkg\master__helipads_POINTS_FOR_AGOL.gpkg

INPUT : master__marinas.gpkg
OUTPUT: master__marinas_POINTS_FOR_AGOL.gpkg
LAYER : marinas_points
After conversion: 19
Point    19
Name: count, dtype: int64
Wrote: scs_osm_project\master_gpkg\master__marinas_POINTS_FOR_AGOL.gpkg

INPUT : master__runways.gpkg
OUTPUT: master__runways_POINTS_FOR_AGOL.gpkg
LAYER : runways_points
After conversion: 9
Point    9
Name: count, dtype: int64
Wrote: scs_osm_project\master_gpkg\master__runways_POINTS_FOR_AGOL.gpkg

INPUT : master__towers.gpkg
OUTPUT: master__tower

### Publish Ready machen

In [11]:
import re
import geopandas as gpd
from pathlib import Path

MASTER_DIR = Path(r"scs_osm_project/master_gpkg")

points_files = sorted(MASTER_DIR.glob("master__*_POINTS_FOR_AGOL.gpkg"))

def is_complex_value(x):
    return isinstance(x, (list, dict, set, tuple))

def fix_field_name(name: str) -> str:
    name = str(name)
    name = re.sub(r"[^0-9A-Za-z_]+", "_", name)
    name = name.strip("_")
    if not name:
        name = "field"
    if name[0].isdigit() or name[0] == "_":
        name = "f_" + name
    return name[:60]

for input_gpkg in points_files:
    layer_key = input_gpkg.stem.replace("master__", "").replace("_POINTS_FOR_AGOL", "")
    output_gpkg = MASTER_DIR / f"master__{layer_key}_PUBLISH_READY.gpkg"
    layer_name = layer_key  # z.B. "towers", "helipads", ...

    print("\n==============================")
    print("INPUT :", input_gpkg.name)
    print("OUTPUT:", output_gpkg.name)
    print("LAYER :", layer_name)
    print("==============================")

    gdf = gpd.read_file(input_gpkg)

    print("Rows:", len(gdf))
    print("CRS:", gdf.crs)
    print("Columns:", len(gdf.columns))

    # 1) Geometrie aufräumen
    gdf = gdf[gdf.geometry.notna()].copy()
    gdf = gdf[~gdf.geometry.is_empty].copy()

    # 2) Komplexe Spalten droppen (list/dict/...)
    drop_cols = []
    for col in gdf.columns:
        if col == "geometry":
            continue
        s = gdf[col]
        if s.dropna().map(is_complex_value).any():
            drop_cols.append(col)

    if drop_cols:
        print("Dropping complex columns:", drop_cols)
        gdf = gdf.drop(columns=drop_cols)

    # 3) Feldnamen ArcGIS-tauglich + eindeutig
    new_cols = {}
    used = set()
    for c in gdf.columns:
        if c == "geometry":
            continue
        fc = fix_field_name(c)
        base = fc
        i = 1
        while fc in used:
            fc = (base[:55] + "_" + str(i))[:60]
            i += 1
        used.add(fc)
        new_cols[c] = fc

    gdf = gdf.rename(columns=new_cols)

    # 4) Datentypen vereinfachen (object -> String)
    for col in gdf.columns:
        if col == "geometry":
            continue
        if gdf[col].dtype == "object":
            gdf[col] = gdf[col].astype(str)

    # 5) CRS: WGS84
    if gdf.crs is None:
        gdf = gdf.set_crs(epsg=4326)
    else:
        gdf = gdf.to_crs(epsg=4326)

    # 6) Schreiben
    gdf.to_file(output_gpkg, layer=layer_name, driver="GPKG")
    print("Wrote:", output_gpkg)
    print("Final columns:", list(gdf.columns)[:20], "...")



INPUT : master__ferry_terminals_POINTS_FOR_AGOL.gpkg
OUTPUT: master__ferry_terminals_PUBLISH_READY.gpkg
LAYER : ferry_terminals
Rows: 21
CRS: EPSG:4326
Columns: 25
Wrote: scs_osm_project\master_gpkg\master__ferry_terminals_PUBLISH_READY.gpkg
Final columns: ['element', 'id', 'amenity', 'building', 'ferry', 'name', 'name_zh_Hant', 'public_transport', 'src_island_id', 'src_island', 'src_country', 'src_group', 'src_layer', 'type', 'name_vi', 'harbour', 'name_zh', 'name_zh_Hans', 'natural', 'name_en'] ...

INPUT : master__helipads_POINTS_FOR_AGOL.gpkg
OUTPUT: master__helipads_PUBLISH_READY.gpkg
LAYER : helipads
Rows: 51
CRS: EPSG:4326
Columns: 19
Wrote: scs_osm_project\master_gpkg\master__helipads_PUBLISH_READY.gpkg
Final columns: ['element', 'id', 'aeroway', 'ref_ourairports', 'src_island_id', 'src_island', 'src_country', 'src_group', 'src_layer', 'surface', 'name', 'name_zh', 'name_zh_Hans', 'name_zh_Hant', 'ref', 'source', 'place', 'note', 'geometry'] ...

INPUT : master__marinas_POINTS

### Workflow um Polygone zu Punkten zu machen für ArcGIS Online

In [2]:
import geopandas as gpd

INPUT_GPKG = r"scs_osm_project/master_gpkg/master__towers.gpkg"
gdf = gpd.read_file(INPUT_GPKG)

print("Rows:", len(gdf))
print(gdf.geom_type.value_counts(dropna=False))
print("CRS:", gdf.crs)


Rows: 189
Point      110
Polygon     79
Name: count, dtype: int64
CRS: EPSG:4326


In [3]:
import geopandas as gpd

INPUT_GPKG  = r"scs_osm_project/master_gpkg/master__towers.gpkg"
OUTPUT_GPKG = r"scs_osm_project/master_gpkg/master__towers_POINTS_FOR_AGOL.gpkg"
LAYER_NAME  = "towers_points"

gdf = gpd.read_file(INPUT_GPKG).copy()

# nur gültige/ vorhandene Geometrie
gdf = gdf[gdf.geometry.notna()].copy()
gdf = gdf[~gdf.geometry.is_empty].copy()

# CRS auf WGS84 (Web-Standard)
if gdf.crs is None:
    # nur wenn du sicher bist, dass es schon lon/lat ist
    gdf = gdf.set_crs(epsg=4326)
else:
    gdf = gdf.to_crs(epsg=4326)

def geom_to_point(geom):
    if geom is None:
        return None
    gt = geom.geom_type
    if gt == "Point":
        return geom
    if gt in ("Polygon", "MultiPolygon"):
        return geom.representative_point()
    if gt in ("LineString", "MultiLineString"):
        # Punkt auf der Linie (Mitte)
        try:
            return geom.interpolate(0.5, normalized=True)
        except Exception:
            return geom.centroid
    # Fallback
    return geom.centroid

gdf["geometry"] = gdf.geometry.apply(geom_to_point)

# nochmal Nulls raus
gdf = gdf[gdf.geometry.notna()].copy()
gdf = gdf[~gdf.geometry.is_empty].copy()

print("After conversion:", len(gdf))
print(gdf.geom_type.value_counts())

# als reiner Point-Layer speichern
gdf.to_file(OUTPUT_GPKG, layer=LAYER_NAME, driver="GPKG")
print("Wrote:", OUTPUT_GPKG)


After conversion: 189
Point    189
Name: count, dtype: int64
Wrote: scs_osm_project/master_gpkg/master__towers_POINTS_FOR_AGOL.gpkg


### Geopackage Dateien für ArcGIS Online optimieren

In [4]:
import re
import pandas as pd
import geopandas as gpd

# ---- ANPASSEN ----
INPUT_GPKG = r"scs_osm_project\master_gpkg\master__towers_POINTS_FOR_AGOL.gpkg"
OUTPUT_GPKG = r"scs_osm_project/master_gpkg/master__towers_PUBLISH_READY.gpkg"
LAYER_NAME = "towers"   # Ziel-Layername im GPKG (einfach halten)
# ------------------

gdf = gpd.read_file(INPUT_GPKG)

print("Rows:", len(gdf))
print("CRS:", gdf.crs)
print("Columns:", len(gdf.columns))

# 1) Geometrie aufräumen: Null/leer raus
gdf = gdf[gdf.geometry.notna()].copy()
gdf = gdf[~gdf.geometry.is_empty].copy()

# 2) (Optional, aber oft hilfreich) Nur Punkte
# Falls dein Helipad-Layer wirklich nur Punkte sein soll:
# gdf = gdf[gdf.geometry.geom_type == "Point"].copy()

# 3) Spalten entfernen, die ArcGIS oft nerven:
# - sehr komplexe OSMnx-Spalten (manchmal dict/list oder gemischte Typen)
def is_complex_value(x):
    return isinstance(x, (list, dict, set, tuple))

drop_cols = []
for col in gdf.columns:
    if col == "geometry":
        continue
    s = gdf[col]
    # wenn irgendwo list/dict/... vorkommt -> weg
    if s.dropna().map(is_complex_value).any():
        drop_cols.append(col)

if drop_cols:
    print("Dropping complex columns:", drop_cols)
    gdf = gdf.drop(columns=drop_cols)

# 4) Feldnamen ArcGIS-tauglich machen:
# Esri: am besten nur [A-Za-z0-9_] und nicht mit Zahl/_ beginnen :contentReference[oaicite:2]{index=2}
def fix_field_name(name: str) -> str:
    name = str(name)
    name = re.sub(r"[^0-9A-Za-z_]+", "_", name)  # Sonderzeichen -> _
    name = name.strip("_")
    if not name:
        name = "field"
    if name[0].isdigit() or name[0] == "_":
        name = "f_" + name
    # optional: sehr lange Namen kürzen (sicherer)
    return name[:60]

new_cols = {}
used = set()
for c in gdf.columns:
    if c == "geometry":
        continue
    fc = fix_field_name(c)
    # Doppelte verhindern
    base = fc
    i = 1
    while fc in used:
        fc = (base[:55] + "_" + str(i))[:60]
        i += 1
    used.add(fc)
    new_cols[c] = fc

gdf = gdf.rename(columns=new_cols)

# 5) Datentypen vereinfachen (Objektspalten -> String)
# (ArcGIS mag klare Typen)
for col in gdf.columns:
    if col == "geometry":
        continue
    if gdf[col].dtype == "object":
        gdf[col] = gdf[col].astype(str)

# 6) CRS: WGS84 ist am „unkompliziertesten“ für Web (EPSG:4326)
# Falls dein CRS None ist oder etwas Exotisches: setze/transformiere.
if gdf.crs is None:
    # nur wenn du sicher bist, dass es schon lon/lat ist:
    gdf = gdf.set_crs(epsg=4326)
else:
    gdf = gdf.to_crs(epsg=4326)

# 7) In ein NEUES GPKG schreiben (am besten nur EIN Layer)
gdf.to_file(OUTPUT_GPKG, layer=LAYER_NAME, driver="GPKG")
print("Wrote:", OUTPUT_GPKG)
print("Final columns:", list(gdf.columns)[:20], "...")


Rows: 189
CRS: EPSG:4326
Columns: 23
Wrote: scs_osm_project/master_gpkg/master__towers_PUBLISH_READY.gpkg
Final columns: ['element', 'id', 'building', 'man_made', 'roof_shape', 'tower_construction', 'tower_type', 'src_island_id', 'src_island', 'src_country', 'src_group', 'src_layer', 'communication_4G', 'communication_mobile_phone', 'building_part', 'monitoring_weather', 'name', 'operator', 'operator_wikidata', 'website'] ...


### (Brauchen wir erstmal nicht) Ein großes Geopackage mit mehreren Layern erstellen

In [5]:
import os
import re
import geopandas as gpd

# =======================
# INPUTS
# =======================
INPUTS = {
    "helipads":         r"scs_osm_project\master_gpkg\master__helipads.gpkg",
    "ferry_terminals":  r"scs_osm_project\master_gpkg\master__ferry_terminals.gpkg",
    "marinas":          r"scs_osm_project\master_gpkg\master__marinas.gpkg",
    "runways":          r"scs_osm_project\master_gpkg\master__runways.gpkg",
    "towers":           r"scs_osm_project\master_gpkg\master__towers.gpkg",
}

# Output GeoPackage (one file, multiple layers)
OUT_MERGED = r"scs_osm_project\master_gpkg\master__ALL.gpkg"
OUT_PUBLISH_READY = r"scs_osm_project\master_gpkg\master__ALL_PUBLISH_READY.gpkg"

# If True: convert everything to POINTS (representative point / mid-point / centroid)
FORCE_POINTS = True

# If True: keep only point geometries after conversion (safer for AGOL if you want points only)
KEEP_ONLY_POINTS = True

# CRS for web/AGOL
TARGET_EPSG = 4326

# =======================
# HELPERS
# =======================
def geom_to_point(geom):
    if geom is None:
        return None
    gt = geom.geom_type
    if gt == "Point":
        return geom
    if gt in ("Polygon", "MultiPolygon"):
        return geom.representative_point()
    if gt in ("LineString", "MultiLineString"):
        try:
            return geom.interpolate(0.5, normalized=True)
        except Exception:
            return geom.centroid
    return geom.centroid

def is_complex_value(x):
    return isinstance(x, (list, dict, set, tuple))

def fix_field_name(name: str) -> str:
    name = str(name)
    name = re.sub(r"[^0-9A-Za-z_]+", "_", name)  # special chars -> _
    name = name.strip("_")
    if not name:
        name = "field"
    if name[0].isdigit() or name[0] == "_":
        name = "f_" + name
    return name[:60]  # safe length

def make_unique_columns(cols):
    new_cols = {}
    used = set()
    for c in cols:
        if c == "geometry":
            continue
        fc = fix_field_name(c)
        base = fc
        i = 1
        while fc in used:
            fc = (base[:55] + "_" + str(i))[:60]
            i += 1
        used.add(fc)
        new_cols[c] = fc
    return new_cols

def prep_for_agol(gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    # 1) geometry cleanup
    gdf = gdf[gdf.geometry.notna()].copy()
    gdf = gdf[~gdf.geometry.is_empty].copy()

    # 2) CRS -> WGS84
    if gdf.crs is None:
        # only do this if you are sure it's already lon/lat
        gdf = gdf.set_crs(epsg=TARGET_EPSG)
    else:
        gdf = gdf.to_crs(epsg=TARGET_EPSG)

    # 3) optionally convert to points
    if FORCE_POINTS:
        gdf["geometry"] = gdf.geometry.apply(geom_to_point)
        gdf = gdf[gdf.geometry.notna()].copy()
        gdf = gdf[~gdf.geometry.is_empty].copy()

    if KEEP_ONLY_POINTS:
        gdf = gdf[gdf.geometry.geom_type == "Point"].copy()

    # 4) drop complex columns (lists/dicts etc.)
    drop_cols = []
    for col in gdf.columns:
        if col == "geometry":
            continue
        s = gdf[col]
        if s.dropna().map(is_complex_value).any():
            drop_cols.append(col)
    if drop_cols:
        gdf = gdf.drop(columns=drop_cols)

    # 5) fix field names (ArcGIS-friendly)
    rename_map = make_unique_columns(gdf.columns)
    gdf = gdf.rename(columns=rename_map)

    # 6) simplify dtypes (object -> string)
    for col in gdf.columns:
        if col == "geometry":
            continue
        if gdf[col].dtype == "object":
            gdf[col] = gdf[col].astype(str)

    return gdf

# =======================
# MERGE (multi-layer gpkg)
# =======================
# remove old outputs to avoid layer overwrite confusion
for out_path in [OUT_MERGED, OUT_PUBLISH_READY]:
    if os.path.exists(out_path):
        os.remove(out_path)

print("Writing merged GeoPackage:", OUT_MERGED)

for layer_name, path in INPUTS.items():
    print("\n---", layer_name, "---")
    gdf = gpd.read_file(path)
    print("Rows:", len(gdf), "| CRS:", gdf.crs)

    # write raw layer into merged gpkg
    gdf.to_file(OUT_MERGED, layer=layer_name, driver="GPKG")
    print("Wrote layer:", layer_name, "to", OUT_MERGED)

print("\n✅ Done: merged file created:", OUT_MERGED)

# =======================
# PUBLISH-READY VERSION
# =======================
print("\nWriting publish-ready GeoPackage:", OUT_PUBLISH_READY)

for layer_name, path in INPUTS.items():
    print("\n---", layer_name, "(publish-ready) ---")
    gdf = gpd.read_file(path)
    gdf2 = prep_for_agol(gdf)

    print("Rows after prep:", len(gdf2))
    print("Geom types:", dict(gdf2.geom_type.value_counts()))
    print("CRS:", gdf2.crs)
    print("Columns:", len(gdf2.columns))

    gdf2.to_file(OUT_PUBLISH_READY, layer=layer_name, driver="GPKG")
    print("Wrote layer:", layer_name, "to", OUT_PUBLISH_READY)

print("\n✅ Done: publish-ready file created:", OUT_PUBLISH_READY)
print("Layers:", list(INPUTS.keys()))


Writing merged GeoPackage: scs_osm_project\master_gpkg\master__ALL.gpkg

--- helipads ---
Rows: 52 | CRS: EPSG:4326
Wrote layer: helipads to scs_osm_project\master_gpkg\master__ALL.gpkg

--- ferry_terminals ---
Rows: 22 | CRS: EPSG:4326
Wrote layer: ferry_terminals to scs_osm_project\master_gpkg\master__ALL.gpkg

--- marinas ---
Rows: 21 | CRS: EPSG:4326
Wrote layer: marinas to scs_osm_project\master_gpkg\master__ALL.gpkg

--- runways ---
Rows: 9 | CRS: EPSG:4326
Wrote layer: runways to scs_osm_project\master_gpkg\master__ALL.gpkg

--- towers ---
Rows: 189 | CRS: EPSG:4326
Wrote layer: towers to scs_osm_project\master_gpkg\master__ALL.gpkg

✅ Done: merged file created: scs_osm_project\master_gpkg\master__ALL.gpkg

Writing publish-ready GeoPackage: scs_osm_project\master_gpkg\master__ALL_PUBLISH_READY.gpkg

--- helipads (publish-ready) ---
Rows after prep: 52
Geom types: {'Point': np.int64(52)}
CRS: EPSG:4326
Columns: 19
Wrote layer: helipads to scs_osm_project\master_gpkg\master__ALL_P